## Pytorch Implementation of [Neural Combinatorial Optimization with Reinforcement Learning](https://arxiv.org/pdf/1611.09940.pdf)

Thanks for [neural-combinatorial-rl-pytorch](https://github.com/pemami4911/neural-combinatorial-rl-pytorch) and [combinatorial optimization with DL/RL](https://github.com/higgsfield/np-hard-deep-reinforcement-learning). Their work helps me understand the theory in paper and fix debugs in my codes. I also used some of their codes. 

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import random
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torch.optim import lr_scheduler

%matplotlib inline

In [2]:
from IPython.display import clear_output
USE_CUDA = False

### Pointer Network

#### Encoder

* One layer of LSTM with (embedding_dim, hidden_dim)
* Initialize h, c with zeros
* Output
    * a sequence of latent memory states $\{enc_i\}_{i=1}^n$, where $\{enc_i\} \in \mathbb{R}^d$, d is embedding dimension
    * h: last layer hidden states, h\[-1\] is the hidden state in the last cell
    * c: last layer cell states, c\[-1\] is the cell state in the last cell

In [3]:
class Encoder(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim):
        super().__init__()
        # use default number of layers is 1
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden_dim = hidden_dim
    
    def forward(self, embedding_x):
        '''
        Args:
        embedding_x (tensor): shape (sequence length, batch, input_dim)
        hidden (tensor): shape (num_layer * num_direction, batch_size, hidden_dim)
        
        Return:
        output (tensor): (sequence length, batch_size, hidden_dim)
        next_hidden (tensor): (num_layer * num_direction, batch_size, hidden_dim)
        '''
        output, next_hidden = self.lstm(embedding_x)
        return output, next_hidden

#### Attention and Glimpse (Pointing and Attending Mechanism)

* Input:
    * a query vector from decoder, $q = dec_i \in \mathbb{R}^d$
    * a set of reference vector from encoder output, $ref = \{enc_i, \dots, enc_k\}$， k could be the sequence length
* Pointing mechanism:
    $$u_i = \begin{cases} v^T \cdot \tanh (W_{ref} \cdot r_t + W_q \cdot q) \hspace{0.2cm} \text{if} \hspace{0.2cm} i \neq \pi(j) \hspace{0.2cm} \text{for all} \hspace{0.2cm} j < i \\
    -\infty\end{cases}  \hspace{0.2cm} \text{for}  \hspace{0.2cm} i = 1, 2, \dots k$$
    where $W_{ref}, W_q \in \mathbb{R}^{d \times d}$ and an attention vector $v \in \mathbb{R}^d$. In practice, the $reference$ input is (batch size $\times$ embedding dimension $\times$ sequence length), we can use convolution weights with shape (embedding dimensions, embedding dimensions) to address 3D data. $v$ initializes with normal distribution between $\left[-\frac{1}{\sqrt{d}}, \frac{1}{\sqrt{d}}\right]$ In the paper, it is $\left[-0.08 (-\frac{1}{\sqrt{128}}), 0.08\right]$. 
    $$A(ref, q; W_{ref}, W_q, v) = softmax(u)$$
    * Improving exploration
        * Softmax temperature
          $$A(ref, q; W_{ref}, W_q, v) = softmax(u / T)$$
          where $T$ is the temperature hyperparameter set to T = 1 during training. $T>1$, $A$ is more smooth, so that is prevent the model from being overconfident
        * Logit clipping (used here)
          $$A(ref, q; W_{ref}, W_q, v) = softmax(C \tanh(u))$$
          where $C$ is the hyperparameter that controls the range of the logits. The paper uses $C = 10$
* Attending mechanism
    $$p = A(ref, q; W_{ref}, W_q, v)$$
    $$G(ref, q; W_{ref}^g, W_q^g, v^g) = \sum_{i = 1}^k r_i p_i$$
    where $G$ is the glimpse function. It can be applied multiple times on the same reference set $ref$:
    $$g_0 = q$$
    $$g_l = G(ref, g_{l-1}; W_{ref}^q, W_q^g, v^g)$$
    The paper says more than once glimpse lead to barely improve thre results. Thus, we choose $\textbf{num_glimpse} = 1$. Glimpse function is implemented in the decoder. 

In [4]:
class Attention(nn.Module):
    def __init__(self, dim, C = 10, use_logit_clip = True, use_cuda = USE_CUDA):
        super().__init__()
        self.w_q = nn.Linear(dim, dim) # for query (batch_size, d)
        self.w_ref = nn.Conv1d(dim, dim, 1, 1) # for reference (batch_size, d, k)
        V = torch.Tensor(dim).float() # to trainable parameters
        if use_cuda:
            V = V.cuda()
        # initialize v by uniform in almost (-0.08, 0.08) from the paper
        self.v = nn.Parameter(V)
        self.v.data.uniform_(- 1. / math.sqrt(dim), 1. / math.sqrt(dim))
        self.tanh = nn.Tanh()
        self.C = C
        self.dim = dim
        self.use_logit_clip = use_logit_clip
    
    def forward(self, encoder_output, query):
        '''
        Args:
        k is sequence length
        encoder_output (tensor): shape is (k, batch_size, dim) from encoder_output
        query (tensor): shape is (batch_size, dim)
        
        Return:
        ref (tensor): shape is (batch_size, dim, k)
        logit (tensor): probability with shape (batch_size, k)
        '''
        batch_size = query.size(0)
        encoder_output = encoder_output.permute(1, 2, 0)
        # make sure ref shape is (batch_size, dim, k)
        
        q = self.w_q(query).unsqueeze(2)
        ref = self.w_ref(encoder_output) # batch_size, hidden_dim, k
        k = ref.size(2)
        expanded_q = q.repeat(1, 1, k)
        expanded_v = self.v.unsqueeze(0).unsqueeze(0).repeat(batch_size, 1, 1)
        # batch matrix multiply (batch_size, sequence_length)
        u = torch.bmm(expanded_v, self.tanh(ref + expanded_q)).squeeze(1)
        
        if self.use_logit_clip:
            logit = self.C * self.tanh(u)
        else:
            logit = u
            
        # return ref for glimpse
        return logit

In [5]:
# mask already search index 
def mask_logit(logit, mask, prev_idx):
    '''
    mask logit probability if they are in previous indice
    Args:
    logit (tensor): from attention output with shape (batch_size, sequence length)
    mask (tensor or None): selected mask
    prev_idx (tensor): None or previous retrieved indice (batch_size, )
    
    Return:
    logit (tensor): same shape with input, but mask elements
    '''
    mask_copy = mask.clone()
    
    batch_size = logit.size(0)
    if prev_idx is not None:
        mask_copy[[b for b in range(batch_size)], prev_idx.data] = 1
        logit[mask_copy] = -np.inf
    
    return logit, mask_copy

#### Decoder

It also maintains its latent memory states $\{dec_i\}_{i=1}^n$, where $dec_i \in \mathbb{R}^d$.
* Input:
    * encoder last cell output: hidden state and cell state
    * decoder input initializes with normal distribution between $\left[-\frac{1}{\sqrt{d}}, \frac{1}{\sqrt{d}}\right]$. It should be trainable parameters.

In [6]:
class Decoder(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, sequence_length, 
                 decoder_type = "sampling", num_glimpse = 1, use_cuda = USE_CUDA):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_glimpse = num_glimpse
        self.sequence_length = sequence_length
        
        # lstm cell weights
        self.lstm_cell = nn.LSTMCell(embedding_dim, hidden_dim)
        
        # pointer and glimpse
        self.pointer = Attention(hidden_dim, use_cuda = use_cuda)
        self.glimpse = Attention(hidden_dim, use_logit_clip=False, use_cuda = use_cuda)
        self.use_cuda = use_cuda
        
        self.softmax = nn.Softmax()
        
        self.decoder_type = decoder_type
    
    def forward(self, decoder_input, embedding_x, hidden, encoder_output):
        '''
        Args:
        decoder_input (tensor): (batch_size, embedding_dim)
        embedding_x (tensor): (k, batch_size, embedding_dim)
        hidden (tuple): (h, c), initially, 
                    (encoder_output[-1], encoder_output[-1]), (batch_size, hidden_dim)
        encoder_output (tensor): encoder output, shape is (k, batch_size, embedding_dim)
        
        Return:
        prob_list (list): list of probability through the sequence 
        index_list (list): list of indice
        hidden (tuple): last layer hidden state and cell state
        '''
        batch_size = decoder_input.size(0)
        seq_len = embedding_x.size(0)
        # save result
        prob_list = []
        index_list = []
        mask = None
        mask = torch.zeros(batch_size, seq_len).byte()
        if self.use_cuda:
            mask = mask.cuda()
        prev_idx = None
        
        embedding_x = embedding_x.permute(1, 0, 2) # to (batch_size, seq_len, embedding_dim)
        ref = encoder_output.permute(1, 2, 0) # to (batch_size, embedding_dim, seq_len)
        for i in range(self.sequence_length):
            h, c = self.lstm_cell(decoder_input, hidden)
            hidden = (h, c)
            g_l = h  # (batch_size, hidden_dim)
            for _ in range(self.num_glimpse):
                logit = self.glimpse(encoder_output, g_l)
                logit, mask = mask_logit(logit, mask, prev_idx)
                p = self.softmax(logit) # (batch_size, seq_len)
                # ref (batch_size, hidden_dim, seq_len)
                g_l = torch.bmm(ref, p.unsqueeze(2)).squeeze(2) # (batch_size, hidden_dim)
            logit = self.pointer(encoder_output, g_l)
            logit_mask, mask = mask_logit(logit, mask, prev_idx)

            if self.decoder_type == "greedy": # for validation
                probs = self.softmax(logit_mask) # batch_size, k
                prev_idx, decoder_input = greedy(probs, embedding_x)
            elif self.decoder_type == "sampling": # for training
                probs = self.softmax(softmax_temperature(logit_mask, T = 1.) )# batch_size, k
                prev_idx, decoder_input = sampling(probs, embedding_x, index_list)
            else:
                raise NotImplementedError
            # record previous index 
            index_list.append(prev_idx)
            # record probability (like lstm output)
            prob_list.append(probs)
        return prob_list, index_list

#### Search Strategies

In [7]:
def greedy(probs, embedding_x):
    '''
    greedy search and return the index with 
    the biggest probability used in validation set
    
    Args:  
    probs (tensor): (batch_size, k)
    embedding_x (tensor): (k, batch_size, embedding_dim)
    
    Return:
    -- new_decoder_input (tensor): selected embedding x with shape (batch_size, embedding_dim)
    as new decoder_input
    -- idx (tensor): selected idx tensor （k,)
    '''
    batch_size = probs.size(0)
    idx = torch.argmax(probs, dim = 1).long()
    new_decoder_input = embedding_x[[x for x in range(batch_size)], idx.data, :]
    return idx, new_decoder_input

def sampling(probs, embedding_x, prev_idx):
    '''
    sampling indice from probability used in train and validation is ok
    
    Args:
    probs (tensor): (batch_size, k)
    embedding_x (tensor): (k, batch_size, embedding_dim)
    prev_idx (list): list of previous index (batch_size, 1), should be LongTensor
    
    Return:
    -- new_decoder_input (tensor): selected embedding x with shape (batch_size, embedding_dim)
    as new decoder_input
    -- idx (tensor): selected idx tensor （k,)
    '''
    batch_size = probs.size(0)
    idx = probs.multinomial(1).squeeze(1).long()
    
    def is_exist(idx, prev_idx):
        for old_idx in prev_idx:
            if old_idx.eq(idx).data.any():
                return True
        return False
    
    while is_exist(idx, prev_idx):
        idx = probs.multinomial(1).squeeze(1).long()  
    
    return idx, embedding_x[[x for x in range(batch_size)], idx.data, :]

def softmax_temperature(logit_mask, T = 2.):
    '''
    Implement softmax temperature strategy
    
    Args:
    logit_mask (batch_size, seq_len)
    T (float): temperature
    '''
    return logit_mask / T

def active_search(self):
    "TO DO"    
    pass

#### Pointer Network Module

Note that we should embed the raw input for the input of the network. Here I spent much time fixing bugs. The embedding for encoders should be shared for a sequence length of LSTM cells inputs, otherwise it will take much time to learn. 

In [8]:
class Embedding(nn.Module):
    def __init__(self, input_size, embedding_size, use_cuda = USE_CUDA):
        super().__init__()
        self.embedding_size = embedding_size
        
        self.embedding = nn.Parameter(torch.FloatTensor(input_size, embedding_size)) 
        self.embedding.data.uniform_(-(1. / math.sqrt(embedding_size)), 1. / math.sqrt(embedding_size))
        
    def forward(self, inputs):
        '''
        Args:
        inputs (batch_size, seq_len, input_size)
        
        Return:
        embedded (batch_size, seq_len, hidden_size)
        '''
        batch_size = inputs.size(0)
        seq_len    = inputs.size(1)
        embedding = self.embedding.repeat(batch_size, 1, 1) # batch_size, input_size, embedding_size
        embedded = []
        for i in range(seq_len):
            # batch multiplication should be same dimensions, so, unsqueeze 1
            embedded.append(torch.bmm(inputs[:, i, :].unsqueeze(1).float(), embedding))
        embedded = torch.cat(embedded, 1)
        return embedded

In [9]:
class PointerNet(nn.Module): 
    
    def __init__(self, embedding_dim, hidden_dim, \
                 seq_length, \
                 decoder_type = "sampling", num_glimpse = 1, use_cuda = USE_CUDA):
        super().__init__()
        
        # define encoder and decoder
        self.encoder = Encoder(embedding_dim, hidden_dim)
        self.decoder = Decoder(embedding_dim, hidden_dim, \
                               seq_length, \
                               decoder_type = decoder_type, \
                               num_glimpse = num_glimpse, use_cuda = use_cuda)

        decoder_input = torch.Tensor(embedding_dim).float()
        self.decoder_input = nn.Parameter(decoder_input) # trainable, default require grad is true
        self.decoder_input.data.uniform_(-1 / math.sqrt(embedding_dim), 1 / math.sqrt(embedding_dim))
        
        # embedding
        self.embedding = Embedding(input_dim, embedding_dim, use_cuda = use_cuda)
    
    def forward(self, x):
        '''
        propagate through the network
        Args:
        x (tensor): (batch_size, k, input_dim), like in paper, embedding dim = d
        
        Return:
        - prob_list (list): length is max length of decoder, 
                with the element shape (batch_size, sequence length), 
        - index_list (list): length is max length of decoder, 
                with the element shape (batch_size,)
        '''
        batch_size = x.size(0)
        embedding_x = self.embedding(x).permute(1, 0, 2)
        encoder_output, (encoder_ht, encoder_ct) = self.encoder(embedding_x)
        
        # last layer output h, c as decoder initial state
        hidden = (encoder_ht.squeeze(0), encoder_ct.squeeze(0)) # (batch_size, hidden_dim)

        decoder_input = self.decoder_input.unsqueeze(0).repeat(batch_size, 1)
        prob_list, index_list = self.decoder(decoder_input, embedding_x, hidden, encoder_output)
        
        return prob_list, index_list

### Critic Network for TSP

Map an input sequence $s$ into a baseline prediction $b_{\theta_v}(s)$, because the objective
$$\mathcal{L}(\theta_v) = \frac{1}{B} \sum_{i=1}^B \left\|b_{\theta_v}(s) - L(\pi_i | s_i) \right\|_2^2$$

* an LSTM encoder: same as the encode in pointer network
* an LSTM process block: performs P steps of computation over the hidden state $h$, update by glimpsing the memory state
* a 2-layer ReLU neural network decoder

In [10]:
class Critic(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, process_iters, use_logit_clip = False, use_cuda = USE_CUDA):
        super().__init__()
        self.encoder = Encoder(embedding_dim, hidden_dim)   
        self.process_block = Attention(hidden_dim, use_logit_clip=use_logit_clip, \
                                       use_cuda = use_cuda) # output is (batch_size, hidden_dim)
        self.decoder = nn.Sequential(
                        nn.Linear(hidden_dim, hidden_dim),
                        nn.ReLU(),
                        nn.Linear(hidden_dim, 1)
                        )
        self.process_iters = process_iters
        self.softmax = nn.Softmax()
        self.embedding = Embedding(input_dim, embedding_dim, use_cuda = use_cuda)
    
    def forward(self, x):
        '''
        Args:
        x (tensor): with shape (k, batch_size, embedding_dim)
        
        Return:
        output (tensor): (batch_size, 1)
        '''
        batch_size = x.size(0)
        embedding_x = self.embedding(x).permute(1, 0, 2)
        encoder_output, (encoder_ht, encoder_ct) = self.encoder(embedding_x)
        ref = encoder_output.permute(1, 2, 0) # to (batch_size, embedding_dim, seq_len)
        g_l = encoder_ht.squeeze(0)
        for p in range(self.process_iters):
            logit = self.process_block(encoder_output, g_l)
            p = self.softmax(logit) # (batch_size, k)
            g_l = torch.bmm(ref, p.unsqueeze(2)).squeeze(2)
        output = self.decoder(g_l)
        return output

#### Combinatorial Model

In [11]:
class Model(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, seq_len, batch_size = 128, process_iters = 3, use_cuda = USE_CUDA):
        super().__init__()
        self.pointer_net = PointerNet(embedding_dim, hidden_dim, seq_len, use_cuda = use_cuda)
        self.critic = Critic(embedding_dim, hidden_dim, process_iters, use_cuda = use_cuda)
        self.batch_size = batch_size
        self.seq_len = seq_len
            
    def forward(self, x):
        '''
        Args:
        x (batch_size, seq_len, input_dim)
        '''
        prob_list, index_list = self.pointer_net(x)
        b = self.critic(x)
        
        pi = []
        probs = []
        for i, index in enumerate(index_list):
            pi_ = x[[j for j in range(self.batch_size)], index.data, :]
            pi.append(pi_)
            prob_ = prob_list[i]
            prob_ = prob_[[j for j in range(self.batch_size)], index.data]
            probs.append(prob_)
        
        L = tour_length(pi)
        log_probs = 0
        for prob in probs:
            log_prob = torch.log(prob)
            log_probs += log_prob
        
        return L, log_probs, pi, index_list, b

### Actor-Critic and Moving Average Training 

#### Calculate tour length

$$L(\pi | s) = \left\| x_{\pi(n)} - x_{\pi(1)} \right\|_2 + \sum_{i=1}^{n-1} \left\|x_{\pi(i)} - x_{\pi(i+1)} \right\|_2$$

In [12]:
# define tour length function as reward
def tour_length(pi, use_cuda = USE_CUDA):
    '''
    calculate the total length of the tour
    Args:
    pi (list): length is sequence length, 
                the element shape is (batch_size, point_size)
    Return:
    tour_len (tensor): (batch_size, 1)
    '''
    
    n = len(pi)
    batch_size = pi[0].size(0)
    tour_len = Variable(torch.zeros(batch_size))
    
    if use_cuda:
        tour_len = tour_len.cuda()
        
    for i in range(n-1):
        tour_len += torch.norm(pi[i+1] - pi[i], p = 2, dim = 1)
    tour_len += torch.norm(pi[n-1] - pi[0], p = 2, dim = 1)
    return tour_len

#### Hyperparameters

In [13]:
# hyperparameter
input_dim = 2
embedding_dim = 128
batch_size = 128
hidden_dim = 128
process_iters = 3
tsp_num = 20
train_size = 1000000
validation_size = 10000
lr = 1e-4
beta = 0.9
num_glimpse = 1

#### Train and Validation Data

In [14]:
def generate_tsp_data():
    '''
    Generate tsp data
    
    Return:
    tsp_data (tensor): shape (1, tsp_num, input_dim)
    '''
    tsp_data = torch.FloatTensor(tsp_num, input_dim).uniform_(0, 1)
    return tsp_data.unsqueeze(0)

train_dataset = [generate_tsp_data() for _ in range(train_size)]
validation_dataset = [generate_tsp_data() for _ in range(validation_size)]
print("train dataset size: ", len(train_dataset))
print("validation dataset size: ", len(validation_dataset))

train dataset size:  1000000
validation dataset size:  10000


In [15]:
# training
class Train(object):
    
    def __init__(self, model, train_set, validation_set, batch_size = 128, max_grad_norm = 1., lr = 1e-4, update_steps = 5000):
        self.model = model
        self.train_set = train_set
        self.validation_set = validation_set
        self.batch_size = batch_size
        self.max_grad_norm = max_grad_norm
        self.optimizer_all = Adam(list(model.critic.parameters()) + list(model.pointer_net.parameters()), lr = lr)
        self.optimizer_pointer = Adam(model.pointer_net.parameters(), lr = lr)
        self.lr_scheduler_pointer = lr_scheduler.MultiStepLR(self.optimizer_pointer, \
                                                     list(range(update_steps, update_steps * 1000, update_steps)), gamma=0.96)
        self.lr_scheduler_all = lr_scheduler.MultiStepLR(self.optimizer_all, \
                                                     list(range(update_steps, update_steps * 1000, update_steps)), gamma=0.96)
        self.mse_loss = nn.MSELoss()
        self.input_dim = 2 # points dimension
        
        self.train_rewards = []
        self.val_rewards = []
    
    def train_and_validation(self, n_epoch, training_steps, use_critic = True):
        moving_average = 0
        for epoch in range(n_epoch):
            for step in range(training_steps):
                training_set = random.sample(self.train_set, batch_size)
                training_set = Variable(torch.cat(training_set).view(self.batch_size, -1, self.input_dim))
                L, log_probs, pi, index_list, b = self.model(training_set)

                log_probs = log_probs.view(-1)
                log_probs[(log_probs < -1000).detach()] = 0.
                
                if not use_critic:
                    if step == 0:
                        moving_average = L.mean()
                    else:
                        moving_average = (moving_average * beta) + ((1. - beta) * L.mean())

                    advantage = L - moving_average             
                    actor_loss = (advantage * log_probs).mean()
                    
                                    
                    self.optimizer_pointer.zero_grad()
                    actor_loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.pointer_net.parameters(), self.max_grad_norm, norm_type=2)
                    self.optimizer_pointer.step()
                    self.lr_scheduler_pointer.step()
                    moving_average = moving_average.detach()
                         
                else:
                    critic_loss = self.mse_loss(b.view(-1), L)
                    advantage = L - b.view(-1)
                    actor_loss = (advantage * log_probs).mean()
                    loss = actor_loss + critic_loss
                    self.optimizer_all.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(list(self.model.critic.parameters()) + \
                                                   list(self.model.pointer_net.parameters()), self.max_grad_norm, norm_type=2)
#                     torch.nn.utils.clip_grad_norm_(, self.max_grad_norm, norm_type=2)
                    self.optimizer_all.step()
                    self.lr_scheduler_all.step()

                self.train_rewards.append(L.mean().data[0])
                
                if step % 10 == 0:
                    self.plot(epoch)
                if step % 100 == 0:
                    val_set = Variable(torch.cat(self.validation_set).view(len(self.validation_set), -1, self.input_dim))
                    L, log_probs, pi, index_list, b = self.model(val_set)
                    self.val_rewards.append(L.mean().data[0])  
                    
                # model save
                if step % 1000 == 0:
                    if use_critic:
                        torch.save(self.model, os.path.join(os.getcwd(), \
                                                            'model_tsp{}_critic.pt'.format(self.model.seq_len)))
                    else:
                        torch.save(self.model, os.path.join(os.getcwd(), \
                                                            'model_tsp{}_mvg_avg.pt'.format(self.model.seq_len)))
    
    def plot(self, epoch):
        clear_output(True)
        plt.figure(figsize=(20,5))
        plt.subplot(131)
        plt.title('train tour length: epoch %s reward %s' % (epoch, self.train_rewards[-1] if len(self.train_rewards) else 'collecting'))
        plt.plot(self.train_rewards)
        plt.grid()
        plt.subplot(132)
        plt.title('val tour length: epoch %s reward %s' % (epoch, self.val_rewards[-1] if len(self.val_rewards) else 'collecting'))
        plt.plot(self.val_rewards)
        plt.grid()
        plt.show() 

#### Moving average reward training, more stable

In [16]:
# moving average reward
moving_average_model = Model(embedding_dim, hidden_dim, seq_len = tsp_num)
if USE_CUDA:
    moving_average_model  = moving_average_model .cuda()
train = Train(moving_average_model , train_dataset, validation_dataset, lr = 1e-4)
train.train_and_validation(6, 10000, use_critic = False)

/home/hexh/miniconda3/envs/marl/lib/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


RuntimeError: masked_fill_ only supports boolean masks, but got mask with dtype unsigned char

#### Actor-Critic training, converge more slowly, not stable, which might be worse sometimes.

I tried learning rate 1e-3 several times, and it might be explode sometimes. Thus, I used same learning rate 1e-4 as moving averages.

In [17]:
# use critic
ac_model = Model(embedding_dim, hidden_dim, seq_len = tsp_num)
if USE_CUDA:
    ac_model  = ac_model.cuda()
train = Train(ac_model, train_dataset, validation_dataset, lr = 1e-4)
train.train_and_validation(6, 10000, use_critic = True)

RuntimeError: masked_fill_ only supports boolean masks, but got mask with dtype unsigned char